# Sentiment Analysis

In [1]:
import pandas as pd
df = pd.read_json( r"C:\Users\Mouni\Downloads\Arts_Crafts_and_Sewing_5.json",lines=True)
df.sample(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
160176,5,True,"12 9, 2017",AX1IDM58UAUEA,B001TNR7VM,NaN,Fargonaut,Smells fantastic.,Five Stars,1512777600,NaN,NaN
184510,5,True,"08 21, 2017",A1AAGEBGJU8IH5,B002N2I1V8,{'Style Name:': ' Tribal Turtles'},Kathy Jones,Cute little turtles for a quick and easy craft.,Quick and easy,1503273600,NaN,NaN
390171,5,True,"04 28, 2018",A237JYWYAWFJI6,B015ZBIME4,NaN,pj,Perfect for art classes,Five Stars,1524873600,NaN,NaN
125998,5,True,"07 2, 2017",A2HD7VAFEK9ZRM,B00172TD5S,{'Size:': ' 1 Pack'},MD,"I really like this needle felting pen, a lot! ...",My favorite felting tool in my collection!,1498953600,NaN,NaN
207792,5,True,"04 4, 2018",A35CJ5NIBXFN63,B004BNAIHW,"{'Size:': ' Size 12', 'Color:': ' Round'}",Elaine Shryock,superior tool,Five Stars,1522800000,NaN,NaN
290837,5,True,"01 10, 2017",A26PSKCF5Q7FQM,B00C1QO3AE,"{'Size:': ' Length-30""-(75Cm)'}",cattylonglegs,I own the complete set of ChiaoGoo Red Lace Tw...,Never Too Many UFOs,1484006400,6,NaN
218689,5,True,"11 1, 2013",A2E0TJ49NACDBA,B004W8PDWQ,{'Format:': ' Home'},DelusionalAngel,I'm not big on pink and a good portion of this...,Definitely one for the girlie girls,1383264000,NaN,NaN
58653,5,True,"04 16, 2015",A1UGKTZ5611JHQ,B000YZ8N46,{'Size:': ' 1 Pack'},Neem,I had used one for years and my grandchildren ...,A must have in your sewing notions,1429142400,NaN,NaN
417198,5,True,"06 18, 2017",AZHNE0UPE2IOX,B01GUYGFNA,NaN,Evelyn A. Gruber,These markers are great and have a very reason...,Great price.,1497744000,NaN,NaN
194821,4,True,"11 5, 2015",A3LIAXZO1OZ6OS,B003E06AQQ,"{'Color:': ' Raspberry', 'Style Name:': ' Pouch'}",Cheryl A.,This is definitely worth the money. Yes the s...,Very handy little bag,1446681600,NaN,NaN


In [2]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize


print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [3]:
##Let's create a dictionary which we can use for scoring our review text

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score

# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score

In [5]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score

In [6]:
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

In [7]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [8]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
